In [1]:
import pandas as pd
import torch
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

ds = load_dataset("sagnikrayc/mctest", "mc160")

In [2]:
# %pip install torch

In [3]:
from huggingface_hub import login
login()

PAD_TOKEN="<|pad|>"
model_name="meta-llama/Meta-Llama-3.1-8B-Instruct"
NEW_MODEL="llama-finetune-mcq-generation"

In [4]:
PAD_TOKEN="<|pad|>"
model_name="meta-llama/Meta-Llama-3.1-8B-Instruct"
NEW_MODEL="llama-finetune-mcq-generation"

In [5]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
from transformers import Trainer, TrainingArguments,AutoTokenizer,AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)
tokenizer.add_special_tokens({"pad_token":PAD_TOKEN})
tokenizer.padding_side="right"

model=AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.resize_token_embeddings(len(tokenizer),pad_to_multiple_of=8)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Embedding(128264, 4096)

In [7]:
ds

DatasetDict({
    train: Dataset({
        features: ['idx', 'question', 'story', 'properties', 'answer_options', 'answer', 'question_is_multiple'],
        num_rows: 280
    })
    validation: Dataset({
        features: ['idx', 'question', 'story', 'properties', 'answer_options', 'answer', 'question_is_multiple'],
        num_rows: 120
    })
    test: Dataset({
        features: ['idx', 'question', 'story', 'properties', 'answer_options', 'answer', 'question_is_multiple'],
        num_rows: 240
    })
})

In [8]:
train_dataset=ds['train']
test_dataset=ds['test']
valid_dataset=ds['validation']

In [9]:
from datasets import Dataset
import pandas as pd
df_train=pd.DataFrame(train_dataset)
df_valid=pd.DataFrame(valid_dataset)
df_test=pd.DataFrame(test_dataset)

df_train=df_train.drop(columns=['idx','properties','question_is_multiple'])
df_valid=df_valid.drop(columns=['idx','properties','question_is_multiple'])
df_test=df_test.drop(columns=['idx','properties','question_is_multiple'])

In [10]:
from textwrap import dedent

def format_example(row: dict):
    prompt = dedent(f"""
    Story:
    {row["story"]}

    Generate a question based on the story above along with four answer options (A, B, C, D) and the correct answer.
    """)

    messages = [
        {
            "role": "system",
            "content": "Use only the story to generate a question, four answer options, and the correct answer.",
        },
        {"role": "user", "content": prompt},
        {
            "role": "assistant",
            "content": dedent(f"""
            Question: {row["question"]}
            Options:
            {row["answer_options"]}
            Correct Answer: {row["answer"]}
            """)
        },
    ]
    
    return tokenizer.apply_chat_template(messages, tokenize=False)


In [11]:
df_train['text']=df_train.apply(format_example,axis=1)
df_valid['text']=df_valid.apply(format_example,axis=1)
df_test['text']=df_test.apply(format_example,axis=1)

In [12]:
print(df_train['text'][0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Use only the story to generate a question, four answer options, and the correct answer.<|eot_id|><|start_header_id|>user<|end_header_id|>

Story:
Tom had to fix some things around the house. He had to fix the door. He had to fix the\newlinewindow. But before he did anything he had to fix the toilet. Tom called over his best\newlinefriend Jim to help him. Jim brought with him his friends Molly and Holly. Tom thought that\newlineJim was going to bring Dolly with him but he didn't. The four of them got to work right\newlineaway. Fixing the toilet was easy. Fixing the door was also easy but fixing the window was\newlinevery hard. The window was stuck and could not be opened. They all pushed on the window\newlinereally hard until finally it opened. Once the window was fixed the four of them made a\newlinedelicious dinner and talked about all of the good work that they 

In [13]:
print(df_train.text.iloc[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Use only the story to generate a question, four answer options, and the correct answer.<|eot_id|><|start_header_id|>user<|end_header_id|>

Story:
Tom had to fix some things around the house. He had to fix the door. He had to fix the\newlinewindow. But before he did anything he had to fix the toilet. Tom called over his best\newlinefriend Jim to help him. Jim brought with him his friends Molly and Holly. Tom thought that\newlineJim was going to bring Dolly with him but he didn't. The four of them got to work right\newlineaway. Fixing the toilet was easy. Fixing the door was also easy but fixing the window was\newlinevery hard. The window was stuck and could not be opened. They all pushed on the window\newlinereally hard until finally it opened. Once the window was fixed the four of them made a\newlinedelicious dinner and talked about all of the good work that they 

In [14]:
def count_tokens(row):
    return len(
        tokenizer(
            row['text'],
            add_special_tokens=True,
            return_attention_mask=False,
        ) ["input_ids"]
    )

df_train['token_count']=df_train.apply(count_tokens,axis=1)
df_valid['token_count']=df_valid.apply(count_tokens,axis=1)
df_test['token_count']=df_test.apply(count_tokens,axis=1)

In [15]:
df_train.to_json("train.json",orient="records",lines=True)
df_valid.to_json("val.json",orient="records",lines=True)
df_test.to_json("test.json",orient="records",lines=True)

In [16]:
dataset=load_dataset(
    "json",
    data_files={"train":"train.json","validation":"val.json","test":"test.json"}
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'story', 'answer_options', 'answer', 'text', 'token_count'],
        num_rows: 280
    })
    validation: Dataset({
        features: ['question', 'story', 'answer_options', 'answer', 'text', 'token_count'],
        num_rows: 120
    })
    test: Dataset({
        features: ['question', 'story', 'answer_options', 'answer', 'text', 'token_count'],
        num_rows: 240
    })
})

# Testing Original Model

In [18]:
from transformers import pipeline
pipe=pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    return_full_text=False
)

In [19]:
def create_test_prompt(data_row):
    prompt = dedent(f"""
    Story:
    {row["story"]}

    Generate a question based on the story above along with four answer options (A, B, C, D) and the correct answer.
    """)

    messages = [
        {
            "role": "system",
            "content": "Use only the story to generate a question, four answer options, and the correct answer.",
        },
        {"role": "user", "content": prompt},
    ]
    
    return tokenizer.apply_chat_template(messages, tokenize=False,add_generation_prompt=True)

In [20]:
row=dataset['test'][0]
prompt=create_test_prompt(row)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Use only the story to generate a question, four answer options, and the correct answer.<|eot_id|><|start_header_id|>user<|end_header_id|>

Story:
Todd is a small boy in the town of Rocksville. Todd thinks his town is the best place in the world, and out of all the places in Rocksville, Todd loves to play in Lake Keet the most. Lake Keet is a small lake with fish, plants, and even little shells. Todd's favorite part was the big rock in the middle of the lake. Todd lives pretty close to the lake, so he gets to go there a lot, but could never to get to the rock in the middle. Todd's dad never lets him swim too far because Todd can't swim very well.\newlineDuring summer, Todd told his dad that he wanted to get to the rock before break ended. Todd's dad told him he had to try his hardest. If he did, then Todd's dad would watch him swim every day. If Todd wanted to try 

In [21]:
%%time
outputs=pipe(prompt)
response=f"""
prediction: {outputs[0]['generated_text']}
"""
print(response)


prediction: Question: What was Todd's dad's condition for watching Todd swim to the rock in the middle of Lake Keet?

A) Todd had to swim to the rock within a day
B) Todd had to practice swimming for a month
C) Todd had to try his hardest and his dad would follow him to ensure his safety
D) Todd had to swim alone without his dad's supervision

Correct answer: C) Todd had to try his hardest and his dad would follow him to ensure his safety

CPU times: user 10.9 s, sys: 703 ms, total: 11.6 s
Wall time: 11.8 s


In [22]:
row['question']

'How long was it before Todd made it to the rock?'

In [23]:
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
# %pip install trl


In [24]:
response_template="<|end_header_id|>"
collator=DataCollatorForCompletionOnlyLM(response_template,tokenizer=tokenizer)

examples=[dataset['train'][0]['text']]
encodings=[tokenizer(e) for e in examples]

dataloader=DataLoader(encodings,collate_fn=collator,batch_size=1)

In [25]:
batch=next(iter(dataloader))
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [26]:
batch['labels']

tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -

# Lora

In [27]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128264, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [28]:
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training 
# %pip install peft

In [31]:
lora_config=LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=["self_attn.q_proj","self_attn.k_proj","self_attn.v_proj","self_attn.o_proj","mlp.gate_proj","mlp.up_proj","mlp.down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model=prepare_model_for_kbit_training(model)
model=get_peft_model(model,lora_config)


In [32]:
model.print_trainable_parameters()

trainable params: 83,886,080 || all params: 8,114,212,864 || trainable%: 1.0338


In [36]:
%pip install tensorboard

In [37]:
output_dir="llama"
%load_ext tensorboard
%tensorboard --logdir "llama/runs"

In [39]:
sft_config=SFTConfig(
    output_dir=output_dir,
    dataset_text_field="text",
    max_seq_length=1024,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    eval_strategy="steps",
    eval_steps=0.2,
    save_steps=0.2,
    logging_steps=10,
    learning_rate=1e-4,
    fp16=True,
    save_strategy="steps",
    warmup_ratio=0.1,
    save_total_limit=2,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    save_safetensors=True,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token":False,
    }
)

trainer=SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    tokenizer=tokenizer,
    data_collator=collator,
)

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

/teamspace/studios/this_studio/.conda/lib/python3.11/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [40]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/teamspace/studios/this_studio/.conda/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/teamspace/studios/this_studio/.conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
14,0.971500,0.745878
28,0.740200,0.726159
42,0.663000,0.724157
56,0.605900,0.728694
70,0.627000,0.728952


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/teamspace/studios/this_studio/.conda/lib/python3.11/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/teamspace/studios/this_studio/.conda/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/teamspace/stud

TrainOutput(global_step=70, training_loss=0.6949858869825091, metrics={'train_runtime': 1596.7793, 'train_samples_per_second': 0.351, 'train_steps_per_second': 0.044, 'total_flos': 1.177138909298688e+16, 'train_loss': 0.6949858869825091, 'epoch': 2.0})

In [41]:
trainer.save_model(NEW_MODEL)


/teamspace/studios/this_studio/.conda/lib/python3.11/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [2]:
import os

# Create the directory if it doesn't exist
offload_folder = "offload_dir"
if not os.path.exists(offload_folder):
    os.makedirs(offload_folder)


In [3]:
from huggingface_hub import login
login()

PAD_TOKEN="<|pad|>"
model_name="meta-llama/Meta-Llama-3.1-8B-Instruct"
NEW_MODEL="llama-finetune-mcq-generation"

In [1]:
!nvidia-smi

Tue Aug 27 05:03:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   33C    P8              38W / 300W |      0MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from transformers import Trainer, TrainingArguments,AutoTokenizer,AutoModelForCausalLM
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training 

import torch
tokenizer = AutoTokenizer.from_pretrained(NEW_MODEL)



In [5]:
!nvidia-smi

Tue Aug 27 05:03:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   31C    P8              15W / 300W |      0MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",  
    offload_folder="offload_dir"  
)
model.resize_token_embeddings(len(tokenizer),pad_to_multiple_of=8)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Embedding(128264, 4096)

In [7]:
!nvidia-smi

Tue Aug 27 05:03:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   34C    P0              57W / 300W |  17710MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
model =PeftModel.from_pretrained(model,NEW_MODEL)


In [9]:
model=model.merge_and_unload()

In [10]:
model.push_to_hub(NEW_MODEL,tokenizer=tokenizer,max_shard_size="4GB")

model-00001-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.17G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Archit1243/llama-finetune-mcq-generation/commit/6922fc2040150f20acb3efa4db1f7a0227f45b3b', commit_message='Upload LlamaForCausalLM', commit_description='', oid='6922fc2040150f20acb3efa4db1f7a0227f45b3b', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
tokenizer.push_to_hub(NEW_MODEL)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Archit1243/llama-finetune-mcq-generation/commit/4c6c8bb6509337239e5d95ff4c7e4d3e725d24a2', commit_message='Upload tokenizer', commit_description='', oid='4c6c8bb6509337239e5d95ff4c7e4d3e725d24a2', pr_url=None, pr_revision=None, pr_num=None)